In [1]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Found existing installation: unsloth 2025.3.14
Uninstalling unsloth-2025.3.14:
  Successfully uninstalled unsloth-2025.3.14
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-07tum5mf
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-07tum5mf
  Resolved https://github.com/unslothai/unsloth.git to commit 6f7c8c6d0a63caaa129cc0bc6b845d5d8b9c81e8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.14-py3-none-any.whl size=194396 sha256=0d25a06428f7d07523073cfe531fe4a28cc82307fc42a754030e37512405b861
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbjjjmx9/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


In [2]:
!pip install datasets transformers trl torch

In [3]:
from unsloth import FastLanguageModel
import torch
import pandas as pd
import numpy as np
from datasets import Dataset
import json
import shutil
from google.colab import files
import os

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.1,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.14 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
import json

with open("/content/conversation_dataset.json", "r", encoding="utf-8") as f:
    try:
        data = json.load(f)
        print("JSON loaded successfully!")
    except json.JSONDecodeError as e:
        print("JSON Error:", e)
print(data[:2])

JSON loaded successfully!
[{'content': 'Detect if the transaction is fraud or not based on the following features:\nV1: 0.9799876052447956\nV2: 0.7736108816346513\nV3: 0.8272829963669474\nV4: 0.2909341549115423\nV5: 0.7660261539744361\nV10: 0.4944353671795013\nV11: 0.28893827145700024\nV12: 0.683667528565137\nV14: 0.6074635728786055\nV16: 0.46803214239167035\nV18: 0.6780358407193249\nV20: 0.579005681819562\nV21: 0.5588535832041219\nV22: 0.48911808543806545', 'role': 'user'}, {'content': 'Result: Not Fraud.\nRisk: Low.\nThe transaction does not show significant anomalies and aligns with normal patterns. Risk Analysis: The transaction has been classified with a Low risk level based on its deviation from normal patterns.', 'role': 'assistant'}]


In [7]:
with open("/content/test_dataset.json", "r", encoding="utf-8") as f:
    try:
        test = json.load(f)
        print("JSON loaded successfully!")
    except json.JSONDecodeError as e:
        print("JSON Error:", e)
test[:2]

JSON loaded successfully!


[{'content': 'Detect if the transaction is fraud or not based on the following features:\nV1: 0.9154139449087113\nV2: 0.7893969698924236\nV4: 0.5381453292946305\nV5: 0.787083036547437\nV11: 0.5462937526675592\nV12: 0.4837217325787546\nV14: 0.2565617705141009\nV15: 0.3990296714886573\nV16: 0.4028888203208292\nV17: 0.7496004625033555\nV19: 0.4803014613160918\nV21: 0.5619298237467011\nV22: 0.5203110886655915\nV24: 0.34308409936748496',
  'role': 'user'},
 {'content': 'Result: Fraud.\nRisk: High.\nThe transaction shows anomalous patterns in variables such as . Risk Analysis: The transaction has been classified with a High risk level based on its deviation from normal patterns.',
  'role': 'assistant'}]

In [13]:
from transformers import TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset

In [17]:
dataset = load_dataset(
    "csv",
    data_files = "/content/balanced.csv",
    split = "train",
)
print(dataset.column_names)
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

['V1', 'V2', 'V3', 'V4', 'V5', 'V10', 'V11', 'V12', 'V14', 'V16', 'V18', 'V20', 'V21', 'V22', 'Class']
{'V1': 0.9799876052447956, 'V2': 0.7736108816346513, 'V3': 0.8272829963669474, 'V4': 0.2909341549115423, 'V5': 0.7660261539744361, 'V10': 0.4944353671795013, 'V11': 0.2889382714570002, 'V12': 0.683667528565137, 'V14': 0.6074635728786055, 'V16': 0.4680321423916703, 'V18': 0.6780358407193249, 'V20': 0.579005681819562, 'V21': 0.5588535832041219, 'V22': 0.4891180854380654, 'Class': 0}


In [18]:
print(dataset.column_names)

['V1', 'V2', 'V3', 'V4', 'V5', 'V10', 'V11', 'V12', 'V14', 'V16', 'V18', 'V20', 'V21', 'V22', 'Class']


In [22]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset,
    merged_prompt = \
        "[[The value of V1 is {V1}.]]"\
        "[[\nThe value of V2 is {V2}.]]"\
        "[[\nThe value of V3 is {V3}.]]"\
        "[[\nThe value of V4 is {V4}.]]"\
        "[[\nThe value of V5 is {V5}.]]"\
        "[[\nThe value of V10 is {V10}.]]"\
        "[[\nThe value of V11 is {V11}.]]"\
        "[[\nThe value of V12 is {V12}.]]"\
        "[[\nThe value of V14 is {V14}.]]"\
        "[[\nThe value of V16 is {V16}.]]"\
        "[[\nThe value of V18 is {V18}.]]"\
        "[[\nThe value of V20 is {V20}.]]"\
        "[[\nThe value of V21 is {V21}.]]"\
        "[[\nThe value of V22 is {V22}.]]",
    conversation_extension = 5,  # Randomly combines conversations into 1! Good for long convos
    output_column_name = "Class",  # Assuming "Class" is the target column (Fraud/Not Fraud)
)


Flattening the indices:   0%|          | 0/984 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/984 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/984 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/984 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/984 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/984 [00:00<?, ? examples/s]

In [23]:
from pprint import pprint
pprint(dataset[0])

{'conversations': [{'from': 'human',
                    'value': 'The value of V1 is 0.9799876052447956.\n'
                             'The value of V2 is 0.7736108816346513.\n'
                             'The value of V3 is 0.8272829963669474.\n'
                             'The value of V4 is 0.2909341549115423.\n'
                             'The value of V5 is 0.7660261539744361.\n'
                             'The value of V10 is 0.4944353671795013.\n'
                             'The value of V11 is 0.2889382714570002.\n'
                             'The value of V12 is 0.683667528565137.\n'
                             'The value of V14 is 0.6074635728786055.\n'
                             'The value of V16 is 0.4680321423916703.\n'
                             'The value of V18 is 0.6780358407193249.\n'
                             'The value of V20 is 0.579005681819562.\n'
                             'The value of V21 is 0.5588535832041219.\n'
                     

In [24]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/984 [00:00<?, ? examples/s]

In [26]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [29]:
chat_template = """Below are details about a financial transaction.
Predict whether it is fraudulent or not based on its characteristics.
Output 1 if it is fraudulent, and 0 if it is not.
>>> Transaction Details:
{INPUT}
>>> Is it fraudulent?
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)


Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/984 [00:00<?, ? examples/s]

In [30]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,  # Reduce batch size
        gradient_accumulation_steps=4,
        warmup_steps=10,  # Increase warmup
        max_steps=40,  # Reduce max steps
        learning_rate=1e-5,  # Lower learning rate
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.1,  # Increase weight decay
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        evaluation_strategy="steps",  # Add evaluation during training
        save_steps=10,  # Save model every 10 steps
        load_best_model_at_end=True,  # Save the best model
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop early if no improvement
)


Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/984 [00:00<?, ? examples/s]

In [31]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 984 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ashrok619 (ashrok619-university-of-east-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.406300
2,2.427100
3,2.406700
4,2.353400
5,2.297400
6,2.251000
7,2.219000
8,2.172500
9,2.134900
10,2.127200


In [40]:
import torch
from transformers import TextStreamer
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Load dataset
test_dataset = pd.read_csv('/content/nearmiss.csv')

# Enable faster inference
FastLanguageModel.for_inference(model)

# Function to get model predictions
def get_prediction(text):
    messages = [{"role": "user", "content": text}]
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    with torch.no_grad():
        output = model.generate(input_ids, max_new_tokens=10, pad_token_id=tokenizer.eos_token_id)

    prediction = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    # Convert model output to binary (0 or 1)
    return int("1" in prediction)  # Adjust logic if necessary

# Prepare test dataset
X_test = test_dataset.drop(columns=["Class"])  # Exclude the target column
y_test = test_dataset["Class"].values          # True labels

# Generate predictions efficiently
y_pred = []
batch_size = 16  # Adjust batch size based on available memory

for i in range(0, len(X_test), batch_size):
    batch = X_test.iloc[i : i + batch_size]
    batch_texts = [", ".join(f"{col}: {row[col]}" for col in X_test.columns) for _, row in batch.iterrows()]

    # Get predictions in batches
    y_pred.extend([get_prediction(text) for text in batch_texts])

# Print evaluation metrics
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=['Not Fraud', 'Fraud']))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

   Not Fraud       0.00      0.00      0.00       492
       Fraud       0.50      1.00      0.67       492

    accuracy                           0.50       984
   macro avg       0.25      0.50      0.33       984
weighted avg       0.25      0.50      0.33       984

Confusion Matrix:
 [[  0 492]
 [  0 492]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [41]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [42]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [43]:
# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.71 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 11.21it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:30<00:00,  4.70s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q4_K_M.gguf
Unsloth: Saved Ollama Modelfile to model/Modelfile


In [44]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3) # Wait for a few seconds for Ollama to load!

In [45]:
print(tokenizer._ollama_modelfile)

FROM {__FILE_LOCATION__}

TEMPLATE """Below are details about a financial transaction. 
Predict whether it is fraudulent or not based on its characteristics.
Output 1 if it is fraudulent, and 0 if it is not.{{ if .Prompt }}
>>> Transaction Details:
{{ .Prompt }}{{ end }}
>>> Is it fraudulent?
{{ .Response }}<｜end▁of▁sentence｜>"""

PARAMETER stop "<|eom_id|>"
PARAMETER stop "<|end_header_id|>"
PARAMETER stop "</think>"
PARAMETER stop "<｜▁pad▁｜>"
PARAMETER stop "<|start_header_id|>"
PARAMETER stop "<｜User｜>"
PARAMETER stop "<|python_tag|>"
PARAMETER stop "<|finetune_right_pad_id|>"
PARAMETER stop "<think>"
PARAMETER stop "<|eot_id|>"
PARAMETER stop "<｜Assistant｜>"
PARAMETER stop "<｜end▁of▁sentence｜>"
PARAMETER stop "<|reserved_special_token_"
PARAMETER temperature 1.5
PARAMETER min_p 0.1


In [46]:
!ollama create finetuned_model -f ./model/Modelfile

gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠦ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠴ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering mode

In [ ]:
import shutil

# Zip the model directory
shutil.make_archive("/content/model_archive", 'zip', "model")


In [ ]:
from google.colab import files

# Download the zip file
files.download("/content/model_archive.zip")
